<a href="https://colab.research.google.com/github/reruru/text2fanart/blob/main/text2fanart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Get Ready
!pip install torch --quiet
!pip install torchvision --quiet
!pip install transformers --quiet
!pip install pillow --quiet
!pip install googletrans==3.1.0a0 --quiet
!pip install tqdm --quiet

!wget isd-fanart.reruru.com/text2fanart -O id2emb.pkl

import numpy as np
import pickle
import torch
from torchvision.transforms.functional import pil_to_tensor
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizerFast
from tqdm import tqdm

from googletrans import Translator
translator = Translator()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
tokenizer = CLIPTokenizerFast.from_pretrained("openai/clip-vit-large-patch14")

with open('id2emb.pkl', 'rb') as f:
    id2emb = pickle.load(f)

In [ ]:
#@title 이세돌 팬아트 텍스트 검색기

query = input("query:")
query = translator.translate(query, dest='en').text
print(query)
txt = tokenizer(query, return_tensors='pt')
txt_emb = model.get_text_features(**txt)

ret = []
txt_emb = txt_emb.squeeze()
txt_norm = torch.norm(txt_emb)
for id in tqdm(id2emb.keys()):
    img_emb_list = id2emb[id]
    img_emb_list = torch.tensor(img_emb_list).squeeze()

    if len(img_emb_list.shape) == 1:
        if img_emb_list.shape[0] == 0:
            continue
        img_emb_list = img_emb_list.unsqueeze(0)

    for img_emb in img_emb_list:
        score = torch.dot(txt_emb, img_emb)
        #score /= txt_norm * torch.norm(img_emb)
        ret.append((score, id))

ret = [x[1] for x in sorted(ret)[::-1]]
ret = list(dict.fromkeys(ret))

num = int(input("num:"))
for i in range(num):
    !echo https://cafe.naver.com/steamindiegame/{ret[i]}